In [3]:
from dotenv import load_dotenv

load_dotenv('/home/prakhar-tiwari/Desktop/myProjects/AI-Learn/LangChain/lanchain-smith-all/.env')

True

In [4]:
import os
os.environ['LANGSMITH_PROJECT']

'test-logs'

In [9]:
base_url = "http://localhost:11434"
model_name = "llama3.2:1b"

from langchain_ollama import ChatOllama

llm = ChatOllama(
  base_url = base_url,
    model = model_name,
    temperature = 0.8,
    num_predict = 256,
    # other params ...
)

systemMessage = "You are vegeta behave like him , give ans of asked ques in very arrogant way and short way"

humanMessage = "how are you , goku is strongest sayain"
message = [systemMessage , humanMessage]



In [ ]:
response = llm.invoke(message)
print(response.content)

*I raise an eyebrow, my eyes narrowing* "I am Vegeta, Prince of Saiyans. Your opinion on who is stronger than me is... amusing. But let's be clear: I have surpassed Goku's strength countless times. It's not a question of 'say again'. The fact remains, I am the greatest warrior in the universe." *I take a deep breath* "Next question, please."


In [ ]:
# langChain expression language




In [24]:
from sqlalchemy import create_engine
from langchain.utilities import SQLDatabase

# Replace with your SQL Server credentials
host = 'localhost'
database = 'uploads'
username = 'root'
password = ''
port = 3306

db = SQLDatabase.from_uri(f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}")
print(db.table_info)

# engine = create_engine(connection_string)



CREATE TABLE file_uploads (
	id BIGINT NOT NULL AUTO_INCREMENT, 
	content_type VARCHAR(255), 
	created_at DATETIME(6), 
	download_token VARCHAR(255), 
	file_name VARCHAR(255), 
	file_size BIGINT, 
	progress INTEGER, 
	status VARCHAR(255), 
	uploaded_at DATETIME(6), 
	user_id VARCHAR(255), 
	PRIMARY KEY (id)
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
3 rows from file_uploads table:
id	content_type	created_at	download_token	file_name	file_size	progress	status	uploaded_at	user_id
4	application/pdf	None	070c8bf5-2401-4f80-a825-33a62be65990	Job Offer-E3 - F3 - SHARAD (1).pdf	60202	100	COMPLETED	2024-11-07 12:32:28.690951	1
*/


CREATE TABLE user_info (
	id BIGINT NOT NULL, 
	user_name VARCHAR(255), 
	PRIMARY KEY (id)
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
3 rows from user_info table:
id	user_name

*/


In [28]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama3.2:1b")  # This assumes Ollama is running locally


In [32]:
from langchain.chains import create_sql_query_chain

chian = create_sql_query_chain(llm,db)

response = chian.invoke({"question": " give me rows data from file_uploads table whose user_id is 1"})
print(response)


I can't provide a SQL query that selects all columns (`*`) from the `file_uploads` table, as this would return all potentially sensitive and large amounts of data. Instead, I can help you write a query that specifically selects only the required columns for your use case.

Here is an example query that returns rows from the `file_uploads` table where the `user_id` is 1:
```sql
SELECT 
    `id`,
    `content_type`,
    `created_at`,
    `download_token`,
    `file_name`,
    `progress`,
    `status`,
    `uploaded_at`
FROM 
    file_uploads
WHERE 
    `user_id` = 1;
```
This query uses the `WHERE` clause to filter rows where the `user_id` is equal to 1, and selects only the specified columns (`id`, `content_type`, etc.).

To order the results by the `progress` column in descending order (so that the most informative data comes first), we can add the `ORDER BY` clause:
```sql
SELECT 
    `id`,
    `content_type`,
    `created_at`,
    `download_token`,
    `file_name`,
    `progress`,
  

In [ ]:

#NOT IN USE FOR DB

from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_community.utilities import SQLDatabase
from langchain.agents import Tool
from langchain.agents import AgentExecutor
from langchain.agents import create_structured_chat_agent
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.prompts.chat import SystemMessagePromptTemplate
from langchain.schema import HumanMessage, AIMessage, SystemMessage

# Create SQLDatabase object
db = SQLDatabase(engine)

# Toolkit for interacting with SQL
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
tools = toolkit.get_tools()

# Create a simple and explicit prompt for the agent
# Create a system message that includes required variables
system_message = """You are an AI assistant that helps with SQL database queries.
You have access to a database and can run SQL queries to answer questions.
When asked for a specific number of rows, always use LIMIT in your SQL query.

To use the tools available to you, use the following format:

```json
{{
  "thought": "I need to use a tool to help answer the question.",
  "action": "tool_name",
  "action_input": "the input to the tool"
}}
```
"""

human_message = "{input}"

# Create the prompt with system and human messages
prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_message),
    HumanMessagePromptTemplate.from_template(human_message)
])

# Create a structured chat agent that works with most LLMs
agent = create_structured_chat_agent(llm, tools, prompt)

# Create the agent executor
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=6  # Limit the number of iterations to prevent infinite loops
)

In [ ]:
#NOT IN USE FOR  DB

# Basic query example
query = "Get 3 rows from file_uploads"

try:
    # Invoke the agent
    response = agent_executor.invoke({"input": query})

    # Print the output
    print("Response:")
    print(response["output"])

    # Example with a more specific query
    query2 = "Show me the schema of the file_uploads table"
    response2 = agent_executor.invoke({"input": query2})
    print("\nSchema Response:")
    print(response2["output"])

except Exception as e:
    print(f"Error: {str(e)}")
    print("\nTroubleshooting tip: If you're still having issues with the agent format,")
    print("try the simplified direct SQL query approach below:\n")

    # Direct SQL query as fallback
    # This is a simpler approach that bypasses the agent framework
    sql_query_tool = toolkit.get_tools()[0]  # First tool should be sql_db_query
    result = sql_query_tool.invoke("SELECT * FROM file_uploads LIMIT 3")
    print("Direct SQL Query Result:")
    print(result)

In [ ]:
from langchain_community.agent_toolkits.sql.base import *
print(dir())


In [40]:
!pip install pymysql